### Reflectors and Projector

In [66]:
import numpy as np
import math as math
sign = np.sign
eye = np.identity
sqrt = math.sqrt
norm = np.linalg.norm
np.set_printoptions(precision=4)

So, let's first of all, take a look at the Householder Reflector

In [2]:
v = np.array([[1], [1]])

The $v$ vector will defines a perpendicular hyper plane which is going to be the reflector, in this case, the reflection plane is just the vector $(-1, 1)$, which lies in the reflection plane. 

In [14]:
P = (eye(2) - 2*(v@v.conj().T)/(v.conj().T@v))
print(P)

[[ 0. -1.]
 [-1.  0.]]


As we can see that the matrix indeed performs a reflection operations on vectors, and the plane of reflection is (-1, 1); by the way, it's a reflector but it's also a projector, it's just different interpretation on the same thing. 

In [68]:
def reflect(x, y):
    """
        projection matrix that project the x onto the plane y lines in.
            * It will return one of the hyperplane with more numerical accurancy, to get the other hyperplane
            the also project to y after the reflection, just take the negative value of it and then it will be down.
        :param x: 
            Column vector 
        :param y: 
            Column vector
    """
    if len(x.shape) == 2 and len(y.shape) == 2:
        y = (y/sqrt(norm(y)))*sqrt(norm(x))  # Y points in y direction and have length of x 
        v = -sign(x[0])*y - x                # Chose the plane to reflec, with better precision
        return (eye(y.shape[0]) - 2*(v@v.conj().T)/(v.conj().T@v))
    else:
        return None
    
def hyperplane_reflect(v):
    """
        Given the vector v, the orthogonal vector the hyperplane, this will produce a reflection that reflects 
        vector with this hyperplane 
        :para v:
            The v vector will have to be a column vector. 
    """
    VHeight = v.shape[0];
    P = (eye(VHeight) - 2*(v@v.conj().T)/(v.conj().T@v))
    return P
    

Now let's try this out with our vector on the householder reflector. 

In [72]:
P = reflect(np.array([[1], [1]]), np.array([[-1], [1]]))
print(P)
print(P@np.array([[1], [1]]))

[[ 1.  0.]
 [ 0. -1.]]
[[ 1.]
 [-1.]]


Let's try out the one vector reflector. 



In [60]:
YAxisReflect = hyperplane_reflect(np.array([[-1], [1]]))
print(YAxisReflect)


[[0. 1.]
 [1. 0.]]


### Designing the Householder Triangularization